In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def parse_raw_position(data):
    data[['alpha', 'delta']] = (
        data['raw position']
        .str.split(expand=True)
        .astype(float)
    )
    
    data.drop('raw position', axis=1, inplace=True)

    return data


In [ ]:
def convert_to_degrees(data):
    columns = ['parallax', 'proper motion alpha', 'proper motion delta',
               'error alpha', 'error delta', 'error parallax',
               'error motion alpha', 'error motion delta']

    for column in columns:
        data[column] = data[column].astype(float) / 3600000.0
        
    return data


In [ ]:
def parse_input_file(filename):
    data = pd.read_csv(filename, delimiter='|', engine='python',
                       skiprows=11,
                       skipfooter=1,
                       index_col=0,
                       header=0,
                       usecols=[1, 4, 7, 9, 10, 11, 12, 13, 14, 15, 16],
                       names=['index', 'magnitude', 'raw position', 'parallax',
                              'proper motion alpha', 'proper motion delta',
                              'error alpha', 'error delta', 'error parallax',
                              'error motion alpha', 'error motion delta'])

    data = parse_raw_position(data)
    data.dropna(inplace=True)
    data = convert_to_degrees(data)
    data['magnitude'] = data['magnitude'].astype(float)
    
    return data
